In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk


import contractions

from string import digits

import re

import pymongo

import gensim

from glob import glob

from bs4 import BeautifulSoup

from bson.objectid import ObjectId

# import spacy

from tqdm import tqdm

import time
from datetime import timedelta

import os, multiprocessing, time
from datetime import timedelta

In [ ]:
nltk.download('stopwords')

# nlp = spacy.load('en_core_web_sm')

In [ ]:
def clean_text(text):
    
    text = text.lower()
    
    remove_digits = str.maketrans('', '', digits)

    res = str(text).translate(remove_digits)

    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)

    text = strip_html_tags(text)
    
    cont = [contractions.fix(tok) for tok in text.split()]
    
    text = ' '.join([word for word in cont if not word.isdigit()])

    text = re.sub(' +',' ',res)
        
    tokenizer = RegexpTokenizer(r'\w+')
    
    tokens = tokenizer.tokenize(text)
    
    clean = [token for token in tokens if token not in stopwords.words('english')]
    
#     stemmer = PorterStemmer()
    
#     clean = [stemmer.stem(word) for word in tokens]
    
    return ' '.join(clean)

In [ ]:
def strip_html_tags(text):
    """remove html tags from text"""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

In [ ]:
conn = pymongo.MongoClient("mongodb://localhost:27017/")

db = conn['BDMSEndSem']

ques_col = db.get_collection('query_answering')

In [ ]:
# one_['clean_text'] = clean_text(one_['accepted_answer'])

# ques_col.update_one({'_id' : ObjectId(one_['_id'])}, one_ ,upsert = False)

# update = {'clean_aa': clean_text(one_['accepted_answer']),
#           'clean_qt': clean_text(one_['question_title']),
#           'clean_qb': clean_text(one_['question_body']),
#           'clean_tg': clean_text(one_['tags'])}


# ques_col.update_one(
#         {'_id': ObjectId(one_['_id'])},
#         { "$set":update })

In [ ]:
n_samp = 1000

D = ques_col.find({}, no_cursor_timeout=True).limit(n_samp)

for i, doc in tqdm(enumerate(D)):
    
    update = {'clean_aa': clean_text(doc['accepted_answer']),
          'clean_qt': clean_text(doc['question_title']),
          'clean_qb': clean_text(doc['question_body']),
          'clean_tg': clean_text(doc['tags'])}
    
    ques_col.update_one(
        {'_id': ObjectId(doc['_id'])},
        { "$set":update })

In [ ]:
from collections import defaultdict
from configparser import ConfigParser
from gensim import corpora, models, similarities

In [ ]:
token_frequency = defaultdict(int)

In [ ]:
n_samp = 10000
docs = ques_col.find({}, no_cursor_timeout=True).limit(n_samp)

In [ ]:
one_ =  ques_col.find_one({}, no_cursor_timeout=True)

In [ ]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

for doc in docs:
    for token in doc['clean_aa'].split(" "):
        token_frequency[token] += 1

In [ ]:
documents = [ [token for token in doc['clean_aa'].split(" ") if token_frequency[token] > 1]
                for doc in docs  ]

In [ ]:
# for doc in documents:
#     doc.sort()

In [ ]:
dictionary = corpora.Dictionary(documents)

In [ ]:
dictionary.compactify()

In [ ]:
dictionary.save('stack_of.dict')

In [ ]:
print(dictionary)

In [ ]:
corpus = [dictionary.doc2bow(doc) for doc in documents]

In [ ]:
corpora.MmCorpus.serialize('stack_of.mm', corpus)

In [ ]:
bow_doc_4310 = corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=100, id2word=dictionary, passes=20, workers=8)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
lda_model_tfidf[corpus[6]]

In [ ]:
unseen_document = 'Applying CSS to Datatables that has NEXT Page'

bow_vector = dictionary.doc2bow(clean_text(unseen_document).split(" "))
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 5)))

In [ ]:


# def clean_text_db(doc):
    
#     update = {'clean_aa': clean_text(doc['accepted_answer']),
#           'clean_qt': clean_text(doc['question_title']),
#           'clean_qb': clean_text(doc['question_body']),
#           'clean_tg': clean_text(doc['tags'])}
        
#     ques_col.update_one(
#         {'_id': ObjectId(doc['_id'])},
#         { "$set":update })
    
#     return 'done'

# # from multiprocessing.pool import ThreadPool as Pool

# pool = multiprocessing.Pool(8)

# start = time.time()

# pool.map(clean_text_db, D)

# end = time.time()

# print("Time Taken:",timedelta(seconds=(end - start)))

In [ ]:
# import multiprocessing
# import os

# multiprocessing.cpu_count()

# import os, multiprocessing, time
# from datetime import timedelta
# def sum_of_cubes(n):
#     return sum(x**3 for x in range(n))

# from multiprocessing.pool import ThreadPool as Pool

# pool = Pool(8)

# start = time.time()
# print(pool.map(sum_of_cubes, range(1000, 100000, 1000)))
# end = time.time()
# print(timedelta(seconds=(end - start)))

In [ ]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in documents]

bow_corpus[4310]

In [ ]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=5, workers=4)

In [ ]:
unseen_document = 'How to add a smooth transition effect'

bow_vector = dictionary.doc2bow(clean_text(unseen_document).split(" "))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 1)))

In [ ]:
lda_model.print_topics()

In [ ]:
lda_model.show_document(9, topn = 5)

In [ ]:
!pip install sklearn

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
lda_model.get_document_topics(corpus[0])

In [ ]:
pip install pyLDAvis.gensim

In [ ]:
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()


In [ ]:
followers_data =  gensimvis.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.display(followers_data)

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.cluster import KMeans
from time import time
import matplotlib.pyplot as plt
import numpy as np
import operator

In [ ]:
topic = None
n_out = 7
n_weight = 5
n_samp=10000
n_components    = 10     # Number of dimension for TruncatedSVD
n_clusters      = 10

In [ ]:
vectorizer  = TfidfVectorizer(max_df=0.9,  min_df=6,
                            max_features=500, use_idf=True,
                            strip_accents='ascii')

In [ ]:
docs = ques_col.find({}, no_cursor_timeout=True).limit(n_samp)

In [ ]:
documents = [ [token for token in doc['clean_aa'].split(" ") if token_frequency[token] > 1]
                for doc in docs  ]

In [ ]:
tokenized   = [ ' '.join(doc) for doc in documents ]

In [ ]:
X = vectorizer.fit_transform(tokenized)

In [ ]:
X = vectorizer.fit_transform(tokenized)

svd  = TruncatedSVD(n_components, random_state= 10)
svdX = svd.fit_transform(X)

In [ ]:
km = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=100, n_init=4,
                verbose=False, random_state= 10)
km.fit(svdX)

In [ ]:
 metrics.silhouette_score(svdX, km.labels_, sample_size=1000)

In [ ]:
terms   = vectorizer.get_feature_names()

In [ ]:
# terms

In [ ]:
len(svd.components_)

In [ ]:
if topic is None:
        for k in range(n_components):
            idx = {i:abs(j) for i, j in enumerate(svd.components_[k])}
            sorted_idx = sorted(idx.items(), key=operator.itemgetter(1), reverse=True)
            weight = np.mean([ item[1] for item in sorted_idx[0:n_weight] ])
            print("T%s)" % k, end =' ')
            for item in sorted_idx[0:n_out-1]:
                print( " %0.3f*%s"  % (item[1] , terms[item[0]]) , end=' ')
            print()
else:
        m = max(svd.components_[topic])
        idx = {i:abs(j) for i, j in enumerate(svd.components_[topic])}
        sorted_idx = sorted(idx.items(), key=operator.itemgetter(1), reverse=True)
        weight = np.mean([ item[1] for item in sorted_idx[0:n_weight] ])
        print("* T %s) weight: %0.2f" % (topic, weight), end=' ')
        for item in sorted_idx[0:n_out-1]:
            print( " %0.3f*%s"  % (item[1] , terms[item[0]]) , end=' ')
        print()

In [ ]:
def plot_clusters(svdX, y_pred, centers):
    plt.style.use('fivethirtyeight')
    f, ax1 = plt.subplots(1, 1, figsize=( 16, 8), facecolor='white')
    ax1.set_xlabel("")
    ax1.set_ylabel("")
    ax1.set_title("K-Means")
    # Only plots the first 2 dimensions of the svdX matrix
    ax1.scatter(svdX[:,0], svdX[:,1], c=y_pred, cmap=plt.cm.Paired, s=45)
    ax1.scatter(centers[:, 0], centers[:, 1], marker='o', c="black", alpha=1, s=150)
    ax1.axis('off')
    plt.show()

In [ ]:
y_pred  = km.predict(svdX)
centers = km.cluster_centers_

plot_clusters(svdX, y_pred, centers)

In [ ]:
from operator import itemgetter

In [ ]:
dictionary = corpora.Dictionary.load("stack_of.dict")
lda_model=models.LdaModel.load('lda_model.lda')
corpus = corpora.MmCorpus("stack_of.mm")

In [ ]:
unseen_document = 'How to add a smooth transition effect'

bow_vector = dictionary.doc2bow(clean_text(unseen_document).split(" "))

vector = lda_model[bow_vector]

vector.sort(key=itemgetter(1), reverse=True)
 
all_topics = lda_model.print_topics()

docs_per_topic = [[] for _ in all_topics]


for doc_id, doc_bow in enumerate(corpus):
    # ...get its topics...
    doc_topics = lda_model.get_document_topics(doc_bow)
    # ...& for each of its topics...
    for topic_id, score in doc_topics:
        # ...add the doc_id & its score to the topic's doc list
        docs_per_topic[topic_id].append((doc_id, score))

In [ ]:
def get_answer(q_text):
    unseen_doc = dictionary.doc2bow(clean_text(q_text).split(" "))
    vector = lda_model[unseen_doc]
    
    vector.sort(key=itemgetter(1), reverse=True)
    
    topic_id = vector[0][0]
    
    doc_id = docs_per_topic[topic_id][0][0]
    
    answer = ques_col.find_one({ "clean_aa": ' '.join(documents[doc_id])})['accepted_answer']
    
    return answer
get_answer('How to add a smooth transition in HTML with CSS?')

In [ ]:
for i in range(10):
    docs_per_topic[i].sort(reverse=True,key=itemgetter(1))

In [ ]:
docs_per_topic[8]

In [ ]:
def get_answer_lsa(q_text):
    unseen_doc=clean_text(q_text).split(" ")
    
    new_vec = dictionary.doc2bow(unseen_doc)
    vec_bow_tfidf = tfidf[new_vec]
    vector = lsi[vec_bow_tfidf]
   

    vector.sort(key=itemgetter(1), reverse=True)
    
    topic_id = vector[0][0]
    
    doc_id = docs_per_topic[topic_id][0][0]
    
    answer = ques_col.find_one({ "clean_aa": ' '.join(documents[doc_id])})['accepted_answer']

    return answer
get_answer_lsa('How to add a smooth transition in HTML with CSS?')

In [ ]:
svd

In [ ]:
lda_model

In [ ]:
from gensim.models import LsiModel

In [ ]:
model = LsiModel(corpus, id2word=dictionary)
vectorized_corpus = model[corpus] 

In [ ]:
model

In [ ]:
tfidf = gensim.models.TfidfModel(corpus, smartirs='npu')
corpus_tfidf = tfidf[corpus]

In [ ]:
lsi = gensim.models.LsiModel(corpus_tfidf, num_topics=10)